In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder # import geocoder

ModuleNotFoundError: No module named 'geocoder'

In [29]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
urltext = requests.get(url).text
data = BeautifulSoup(urltext, 'lxml')


columnlabel = ['Postalcode','Borough','Neighborhood']
trnt = pd.DataFrame(columns = columnlabel)

content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text[:-1]
            i = i + 1
        elif i == 1:
            borough = td.text[:-1]
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    trnt = trnt.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

trnt = trnt[trnt.Borough!='Not assigned']
trnt = trnt[trnt.Borough!= 0]
trnt.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,trnt.shape[0]):
    if trnt.iloc[i][2] == 'Not assigned':
        trnt.iloc[i][2] = trnt.iloc[i][1]
        i = i+1
                                 
df = trnt.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
output = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
output.rename(columns={'Postalcode': 'Postal Code'}, inplace=True)
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
output = output.merge(geo_df, how='left')
output

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
